In [ ]:
!pip install openai requests

import openai
import requests

OPENAI_API_KEY = "your-api-key-here"
DEEPSEEK_API_KEY = "your-deepseek-api-key-here"

if not OPENAI_API_KEY or not DEEPSEEK_API_KEY:
    raise ValueError("API keys must be set before running the script.")

def chatgpt_parse(prompt):
    try:
        client = openai.OpenAI(api_key=OPENAI_API_KEY)
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "system", "content": "You are a helpful assistant."},
                      {"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content
    except openai.AuthenticationError:
        return "ChatGPT API Error: Invalid API Key."
    except openai.RateLimitError:
        return "ChatGPT API Error: Rate limit exceeded."
    except openai.OpenAIError as e:
        return f"ChatGPT API Error: {str(e)}"
    except Exception as e:
        return f"Unexpected Error: {str(e)}"

def deepseek_parse(prompt):
    try:
        url = "https://api.deepseek.com/v1/chat/completions"
        headers = {"Authorization": f"Bearer {DEEPSEEK_API_KEY}", "Content-Type": "application/json"}
        payload = {"model": "deepseek-chat", "messages": [{"role": "user", "content": prompt}]}
        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()
        result = response.json()
        return result.get("choices", [{}])[0].get("message", {}).get("content", "No response")
    except requests.exceptions.HTTPError as e:
        return f"DeepSeek API Error: HTTP Error {response.status_code} - {str(e)}"
    except requests.exceptions.ConnectionError:
        return "DeepSeek API Error: Failed to connect to the server."
    except requests.exceptions.Timeout:
        return "DeepSeek API Error: Request timed out."
    except requests.exceptions.RequestException as e:
        return f"DeepSeek API Error: {str(e)}"
    except Exception as e:
        return f"Unexpected Error: {str(e)}"

if __name__ == "__main__":
    prompt = "Extract key points from the following text: Artificial Intelligence is transforming industries by automating tasks, enhancing decision-making, and enabling new innovations."
    parsed_output_chatgpt = chatgpt_parse(prompt)
    parsed_output_deepseek = deepseek_parse(prompt)

    print("\nChatGPT Parsed Output:")
    print(parsed_output_chatgpt)

    print("\nDeepSeek Parsed Output:")
    print(parsed_output_deepseek)
